In [1]:
import pandas as pd
import numpy as np 

#Make numpy values easier to read
np.set_printoptions(precision = 3, suppress=True)

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import layers

2025-06-04 17:54:22.443741: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/alex/Documents/NeuralNetwork/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('data/titanic.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(['Name','PassengerId','Ticket'],axis = 1, inplace= True)

In [5]:
df = df[['Survived','Sex','Age','SibSp','Parch','Fare','Pclass','Cabin','Embarked']]

In [6]:
X = df[['Sex','Age','SibSp','Parch','Fare','Pclass','Cabin','Embarked']]
y = df[['Survived']]

In [7]:
#Create a symbolic input
input = tf.keras.Input(shape=(), dtype=tf.float32)

# Perform a calculation using the input
result = 2*input +1

# the result doesn't have a value
result

<KerasTensor shape=(None,), dtype=float32, sparse=False, ragged=False, name=keras_tensor_2>

In [8]:
calc = tf.keras.Model(inputs=input, outputs= result)


In [9]:
print(calc(np.array([1])).numpy())
print(calc(np.array([2])).numpy())


[3.]
[5.]


To build the preprocessing model, start by building a set of symbolic `tf.keras.Input` objects, matching the names and data-types of the CSV columns

In [10]:
inputs = {}
for name, column in X.items():
    dtype = column.dtype
    if dtype == object:
        dtype = tf.string
    else:
        dtype = tf.float32

    inputs[name] = tf.keras.Input(shape=(1,), name = name, dtype= dtype)

inputs


{'Sex': <KerasTensor shape=(None, 1), dtype=string, sparse=False, ragged=False, name=Sex>,
 'Age': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, ragged=False, name=Age>,
 'SibSp': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, ragged=False, name=SibSp>,
 'Parch': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, ragged=False, name=Parch>,
 'Fare': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, ragged=False, name=Fare>,
 'Pclass': <KerasTensor shape=(None, 1), dtype=float32, sparse=False, ragged=False, name=Pclass>,
 'Cabin': <KerasTensor shape=(None, 1), dtype=string, sparse=False, ragged=False, name=Cabin>,
 'Embarked': <KerasTensor shape=(None, 1), dtype=string, sparse=False, ragged=False, name=Embarked>}

The first step in your preprocessing logic is to concatenate the numeric inputs together, and run them through a normalization layer:

In [11]:
numeric_inputs = {name: input for name, input in inputs.items()
                 if input.dtype ==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = layers.Normalization()
norm.adapt(np.array(df[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

<KerasTensor shape=(None, 5), dtype=float32, sparse=False, ragged=False, name=keras_tensor_4>

Collect all the symbolic preprocessing results, to concatenate them later:

In [12]:
preprocessed_inputs = [all_numeric_inputs]

In [ ]:

preprocessed_inputs

[<KerasTensor shape=(None, 5), dtype=float32, sparse=False, ragged=False, name=keras_tensor_4>]

For the String inputs, use the `tf.keras.layers.Stringlookup` function to map from strings to integer indices in a vocabulary. Next, use `tf.keras.layers.CategoryEncoding` to convert the indexes into `float32` data appropriate for the model

In [33]:
for name, input in inputs.items():
    if input.dtype == tf.float32:
        continue
    else:
        print(input)

    lookup = layers.StringLookup(vocabulary=np.unique(X[name].astype(str)))
    one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size()) 

x = lookup(input)
x = one_hot(input)
preprocessed_inputs.append(x)

<KerasTensor shape=(None, 1), dtype=string, sparse=False, ragged=False, name=Sex>
<KerasTensor shape=(None, 1), dtype=string, sparse=False, ragged=False, name=Cabin>
<KerasTensor shape=(None, 1), dtype=string, sparse=False, ragged=False, name=Embarked>


With the collection of `inputs` and `preprocessed_inputs` , you can concatenate all the preprocessed inputs together, and build a model that handles the preprocessing:

In [36]:
preprocessed_inputs

[<KerasTensor shape=(None, 5), dtype=float32, sparse=False, ragged=False, name=keras_tensor_4>,
 <KerasTensor shape=(None, 5), dtype=float32, sparse=False, ragged=False, name=keras_tensor_6>,
 <KerasTensor shape=(None, 5), dtype=float32, sparse=False, ragged=False, name=keras_tensor_10>]

In [27]:
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

titanic_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

tf.keras.utils.plot_model(model = titanic_preprocessing,  rankdir= 'LR', dpi=72, show_shapes= True)

ValueError: `inputs` not connected to `outputs`

In [30]:
preprocessed_inputs_cat

<KerasTensor shape=(None, 10), dtype=float32, sparse=False, ragged=False, name=keras_tensor_8>